In [2]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
import requests
from lmfit import minimize, Parameters, Parameter, report_fit
pio.renderers.default = "notebook"
%matplotlib inline
#plt.style.use('ggplot')


In [16]:
pip install numpy==1.23.0

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [101]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import r2_score#R square
def mape(z, upcoming_days_prediction):
    return np.mean(np.abs((z - upcoming_days_prediction) / z)) * 100
 
def smape(z, upcoming_days_prediction):
    return 2.0 * np.mean(np.abs(z - upcoming_days_prediction) / (np.abs(z) + np.abs(upcoming_days_prediction))) * 100

In [102]:
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

In [103]:
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, ToggleButton, ToggleButtons

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [104]:
def ode_model(z, t,  beta, beta1, beta2,  k1,k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """

    S,S1, E_0,E_V, I_0,I_V, R_0,R_V,R3,R4,D,I_C= z
    N = S+S1+ E_0+E_V+ I_0+I_V+D+R_0+R_V+R3+R4+D
    #IPV=beta*S*I/N+beta1*S*P/N+beta2*S*V/N
    dSdt = -beta*S*I_0/N-beta1*S*I_V/N-alpha1*S
    dE_0dt = beta*S*I_0/N+beta*R4*I_0/N - k1*E_0
    dI_Cdt=k1*E_0+k2*E_V
    dI_0dt = k1*E_0 - (gama1+deta1)*I_0
    dR_0dt = gama1*I_0-yita2*R_0
    dR3dt = yita2*R_0-beta1*R3*I_V/N-alpha1*R3
    dS1dt = alpha1*(S+R3+R4)-beta2*(S1-alpha1*R4)*I_V/N
    dE_Vdt = beta1*S*I_V/N+beta2*(S1-alpha1*R4)*I_V/N+beta1*R3*I_V/N - k2*E_V
    dI_Vdt=k2*E_V-(deta2+gama2)*I_V
    dR4dt =yita1*R_V-beta*R4*I_0/N-alpha1*R4
    dR_Vdt=gama2*I_V-yita1*R_V
    dDdt=deta1*I_0+deta2*I_V
    return [dSdt,dS1dt, dE_0dt,dE_Vdt, dI_0dt,dI_Vdt, dR_0dt,dR_Vdt,dR3dt,dR4dt,dDdt,dI_Cdt]

In [105]:
def ode_solver(t, initial_conditions, params):
    initS1,initE_0,initE_V, initI_0,initI_V,initR_0,initR_V,initR3,initR4,  initN,  initD,initI_C= initial_conditions
    beta, beta1, beta2, k1,k2, deta1, deta2,gama1,gama2,alpha1,yita1,yita2 = params['beta'].value,params['beta1'].value,params['beta2'].value,params['k1'].value,params['k2'].value,params['deta1'].value,params['deta2'].value,params['gama1'].value,params['gama2'].value,params['alpha1'].value,params['yita1'].value,params['yita2'].value
    initS = initN-initS1 - initE_0-initE_V-initI_0-initI_V-initR_0-initR_V-initR3-initR4-initD
    
    res = odeint(ode_model, [initS,initS1, initE_0,initE_V,initI_0,initI_V,initR_0,initR_V,initR3,initR4,initD,initI_C], t, args=(beta, beta1, beta2, k1, k2,deta1, deta2,gama1, gama2,alpha1,yita1,yita2))
    return res

In [189]:
data1 = pd.read_excel('/Volumes/TOSHIBA EXT/percountrydata/Bahrain daily cases.xlsx')
z=data1['cases']
x=data1['cumulative_cases']



In [196]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 1700000
# S0 = 966000000
initS1 = 1
initI_C = 278065
initE_0 = 1
initE_V = 0
initI_0 = 341
initI_V = 1
initR_0 =276355
initR_V = 0
initR3 = 0
initR4 = 0
initD = 1394

beta = 1.26329615
beta1 = 2.11158139
beta2 =  0.30679778 
#alpha1=alpha1
k1 = 0.25000000
k2=  0.37000000
deta1 =0.10000000
deta2 =  0.10000000

gama1 =  0.30000000
gama2 = 0.29997115
alpha1=0.04642130
yita1=0.03305947
yita2= 0.08749564
days =140

params = Parameters()
params = Parameters()
params.add('beta', value=beta, min=0.7, max=7)
params.add('beta1', value=beta1, min=1.5, max=7)
params.add('beta2', value=beta2, min=0.3, max=4)

params.add('k1', value=k1, min=0.15, max=0.25)
params.add('k2', value=k2, min=0.3, max=0.37)
params.add('deta1', value=deta1, min=0, max=0.1)
params.add('deta2', value=deta2, min=0, max=0.1)

params.add('gama1', value=gama1, min=0.05, max=0.3)
params.add('gama2', value=gama2, min=0.05, max=0.3)
params.add('alpha1', value=alpha1, min=0, max=0.3)
params.add('yita1', value=yita1, min=0, max=0.2)
params.add('yita2', value=yita2, min=0, max=1)



In [108]:
def main(initI_C,initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V, initR3,initR4, initN, initD ,beta, beta1,beta2, k1, k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2, days, param_fitting):
    initial_conditions = [initI_C,initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V,initR3,initR4, initN ,initD,]
    params['beta'].value,params['beta1'].value,params['beta2'].value,params['k1'].value,params['k2'].value,params['deta1'].value,params['deta2'].value,params['gama1'].value,params['gama2'].value,params['alpha1'].value,params['yita1'].value,params['yita2'].value = [beta, beta1, beta2,  k1,k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2]
    
    tspan = np.arange(0, days, 1)
    #beta=np.arange(0, 0.01, 0.5)
    sol = ode_solver(tspan, initial_conditions, params)
    S,S1,E_0,E_V, I_0,I_V, R_0 ,R_V ,R3 ,R4, D, I_C= sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4], sol[:, 5], sol[:, 6], sol[:, 7], sol[:, 8],sol[:, 9],sol[:, 10],sol[:, 11]
    '''for t in range (0,len(I_C)):
        print(type(I_C))'''

    print(max(k1*E_0))
            #print(alpha1)
            #print(beta2)

In [200]:
def main(initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V, initR3,initR4, initN, initD  ,initI_C  ,beta, beta1,beta2, k1, k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2, days, param_fitting):
    initial_conditions = [initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V,initR3,initR4, initN ,initD,initI_C]
    params['beta'].value,params['beta1'].value,params['beta2'].value,params['k1'].value,params['k2'].value,params['deta1'].value,params['deta2'].value,params['gama1'].value,params['gama2'].value,params['alpha1'].value,params['yita1'].value,params['yita2'].value = [beta, beta1, beta2,  k1,k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2]
    
    tspan = np.arange(0, days, 1)
    
    sol = ode_solver(tspan, initial_conditions, params)

   
    '''print('MSE',np.square(np.subtract(x[120:127],(sol[:, 11][120:127]))).mean())
    print('RMSE: ', np.sqrt(np.mean((x[120:127]- (sol[:, 11][120:127]))**2)))
    print('RMAE',np.sqrt(metrics.mean_squared_error(x[120:127], (sol[:, 11][120:127]))))
    
    print( 'MAE',np.mean(np.abs(x[120:127]- (sol[:, 11][120:127]))))
    print('MAPE',mape(x[120:127], (sol[:, 11][120:127])))
    print('SMAPE',smape(x[120:127],(sol[:, 11][120:127])))
    print('R2',r2_score(x[120:127],(sol[:, 11][120:127])))''' 
    '''for i in range(0, len(tspan)):
        
        if tspan[i]<50:
            params['alpha1'].value=0
        else:
            params['alpha1'].value=0.1''' 


    #print(alpha1)
    
    S,S1,E_0,E_V, I_0,I_V, R_0 ,R_V ,R3 ,R4, D, I_C= sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4], sol[:, 5], sol[:, 6], sol[:, 7], sol[:, 8],sol[:, 9],sol[:, 10],sol[:, 11]
    
    #print(I_V/(I_0+I_V))
    
    #print(max(I_0))
    '''temp = []
    i = 0
    cout = 0
    for t in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            i += (I_V[t]/(I_V[t]+I_0[t]))
        if cout == 14:
            i += (I_V[i]/(I_V[i]+I_0[i]))
            temp.append(i)
            cout = 0
            i = 0
    G = np.array(temp)
    print(G)''' 
    
    temp = []
    i = 0
    cout = 0
    for t in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            i += (I_V[t]+I_0[t])
        if cout == 14:
            i += (I_V[t]+I_0[t])
            temp.append(i)
            cout = 0
            i = 0
    Qemp = []
    j = 0
    cout = 0
    for m in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            j += (I_V[m])
        if cout == 14:
            j += (I_V[m])
            Qemp.append(j)
            cout = 0
            j = 0
    '''G = np.array(temp)
    M = np.array(Qemp)'''
    

    '''Qemp = []
    j = 0
    cout = 0
    for m in range(0, len(sol[:, 6])):
        cout += 1
        if cout < 14:
            j += (sol[:, 6][m])
        if cout == 14:
            j += (sol[:, 6][m])
            Qemp.append(j)
            cout = 0
            j = 0'''
    
    G = np.array(temp)
    M = np.array(Qemp)
    
    #print(M/G)
    
    '''qemp = []
    j = 0
    cout = 0
    for m in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            j += (I_V[m])
        if cout == 14:
            j += (I_V[m])
            temp.append(j)
            cout = 0
            j = 0
    M = np.array(qemp)
    print(M)'''   
    
    
    #df = pd.DataFrame (I_C)
    #print(k1*E_0+k2*E_V)
    #filepath = '/Users/mac/Desktop/trans.xlsx'
    #df.to_excel(filepath, index=False)
  # Create traces
    fig = go.Figure()
    



    
    if not param_fitting:
        #fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers',name='Susceptible'))
        #fig.add_trace(go.Scatter(x=tspan, y=S1, mode='lines+markers',name='Susceptible1'))
        #fig.add_trace(go.Scatter(x=tspan, y=E_0, mode='lines+markers', name='Exposed_0'))
        #fig.add_trace(go.Scatter(x=tspan, y=E_V, mode='lines+markers', name='Exposed_V'))
        #fig.add_trace(go.Scatter(x=tspan, y=I_0, mode='lines+markers', name='Infected0'))
        fig.add_trace(go.Scatter(x=tspan, y=k1*E_0, mode='lines+markers', name='delta'))
        fig.add_trace(go.Scatter(x=tspan, y=k2*E_V, mode='lines+markers', name='Omicron'))
        #fig.add_trace(go.Scatter(x=tspan, y=k2*E_V, mode='lines+markers', name='Omicron'))
        #fig.add_trace(go.Scatter(x=tspan, y=R3, mode='lines+markers',name='Recovered0S'))
        #fig.add_trace(go.Scatter(x=tspan, y=R_0, mode='lines+markers',name='Recovered0'))
        #fig.add_trace(go.Scatter(x=tspan, y=R_V, mode='lines+markers',name='RecoveredV'))
        #fig.add_trace(go.Scatter(x=tspan, y=R4, mode='lines+markers',name='RecoveredVS'))
    #fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines',name='mj'))
    fig.add_trace(go.Scatter(x=tspan, y=k1*E_0+k2*E_V, mode='lines', name='Fitting', line = dict ( width = 5 , color = 'lightcoral' )))
    #fig.add_trace(go.Scatter(x=tspan, y=I_C, mode='lines', name='Fitting',line = dict ( width = 5 , color = 'lightcoral' )))
        #
        #
        
        
    
        
        
    #fig.add_trace(go.Scatter(x=tspan, y=k1*E_0+k2*E_V, mode='lines+markers',line = dict(width=1,color='black'),name='Fitting curve',marker=dict(color='white',size = 8 , line = dict ( width = 1 , color = 'black'))))
        

    '''fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines+markers',line = dict(width=1,color='black'),name='Fitting curve',marker=dict(color='white', size = 8 , line = dict ( width = 1 , color = 'black'))))'''
      
    if param_fitting:
        '''fig.add_trace(go.Scatter(x=tspan, y=x, mode='markers',\
                             name='Real data', line = dict(dash='dash')))
        fig.update_traces ( marker = dict ( size = 8 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))'''
        fig.add_trace(go.Scatter(x=tspan, y=z, mode='markers',\
                             name='Real data', line = dict(dash='dash'),marker=dict ( color='LightSkyBlue',
        size = 8 , line = dict ( width = 1 , color = 'black' ))))
        #fig.add_trace(go.Scatter(x=tspan, y=m, mode='lines', name='cumu',line = dict ( width = 5 , color = 'orange' )))
    
        
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 31
    
    # Edit the layout
    fig.update_layout(font=dict(size=20),
                        
                       xaxis_title='Day',
                       yaxis_title='Number of New Cases',
                       #yaxis_title='Total Cases',
                       title_x=0.5,width=900, height=600, template = 'simple_white',legend=dict(font=dict(size=15),yanchor="top",y=0.99,xanchor="left",x=0.01,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>29 Nov 2021</b>', '<b>30 Dec 2021</b>','<b>30 Jan 2022</b> ', '<b>2 Mar 2022</b>','<b>2 Apr 2022</b>', '<b>3 May 2022</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("images/S.2daily.JPG",scale=2)
    fig.write_image("/Users/mac/Desktop/case&per/Bahrain daily.JPG",scale=2)
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)


    fig.show()




In [201]:
interact(main,
        initS1=IntSlider(min=0, max=100000, step=1, value=initS1, description='initS1', style=style, layout=slider_layout),
        initI_C=IntSlider(min=0, max=100000000, step=100, value=initI_C, description='initI_C', style=style, layout=slider_layout),
          
        initE_0=IntSlider(min=0, max=10000000, step=1, value=initE_0, description='initE_0', style=style, layout=slider_layout),
        initE_V=IntSlider(min=0, max=100000, step=1, value=initE_V, description='initE_V', style=style, layout=slider_layout),
        
        initI_0=IntSlider(min=0, max=100000000, step=10, value=initI_0, description='initI_0', style=style, layout=slider_layout),
        initI_V=IntSlider(min=0, max=100000, step=10, value=initI_V, description='initI_V', style=style, layout=slider_layout),
        
        initR_0=IntSlider(min=0, max=10000000, step=10, value=initR_0, description='initR_0', style=style, layout=slider_layout),
        initR_V=IntSlider(min=0, max=100000, step=10, value=initR_V, description='initR_V', style=style, layout=slider_layout),
        initR3=IntSlider(min=0, max=1000000, step=100, value=initR3, description='initR_0S', style=style, layout=slider_layout),
        initR4=IntSlider(min=0, max=1000000, step=100, value=initR4, description='initR_VS', style=style, layout=slider_layout),
        
        initD=IntSlider(min=0, max=100000, step=10, value=initD, description='initD', style=style, layout=slider_layout),
        initN=IntSlider(min=0, max=320000000, step=1000, value=initN, description='initN', style=style, layout=slider_layout),
        #initIPV=IntSlider(min=0, max=100000, step=10, value=initN, description='initIPV', style=style, layout=slider_layout),
        beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
        beta1=FloatSlider(min=0, max=14, step=0.01, value=beta1, description='beta1', style=style, layout=slider_layout),
        beta2=FloatSlider(min=0, max=4, step=0.01, value=beta2, description='beta2', style=style, layout=slider_layout),
        
        k1=FloatSlider(min=0, max=1, step=0.01, value=k1, description='k1', style=style, layout=slider_layout),
        k2=FloatSlider(min=0, max=1, step=0.01, value=k2, description='k2', style=style, layout=slider_layout),
        deta1=FloatSlider(min=0, max=1, step=0.01, value=deta1, description='deta1', style=style, layout=slider_layout),
        deta2=FloatSlider(min=0, max=1, step=0.01, value=deta2, description='deta2', style=style, layout=slider_layout),
        
        gama1=FloatSlider(min=0, max=1, step=0.01, value=gama1, description='gama1', style=style, layout=slider_layout),
        gama2=FloatSlider(min=0, max=1, step=0.01, value=gama2, description='gama2', style=style, layout=slider_layout),
        alpha1=FloatSlider(min=0, max=1, step=0.01, value=alpha1, description='alpha1', style=style, layout=slider_layout),
        yita1=FloatSlider(min=0, max=1, step=0.001, value=yita1, description='yita1', style=style, layout=slider_layout),
        yita2=FloatSlider(min=0, max=1, step=0.01, value=yita2, description='yita2', style=style, layout=slider_layout),
        
        days=IntSlider(min=1, max=4000, step=1, value=days, description='Days', style=style, layout=slider_layout),
        param_fitting=ToggleButton(value=False, description='Fitting Mode', disabled=False, button_style='', \
            tooltip='Click to show fewer plots', icon='check-circle')
              );
        
        

               

interactive(children=(IntSlider(value=1, description='initS1', layout=Layout(width='99%'), max=100000, style=S…

In [158]:
def error(params, initial_conditions, tspan, data):
    sol = ode_solver(tspan, initial_conditions, params)

    #sol_=k1*sol[:, 2]+k2*sol[:, 3]
    #SOL=np.hstack((C,sol_))
    #print(SOL)
    
    #print(M/G)
    sol_=k1*sol[:, 2]+k2*sol[:, 3]
    #return (sol_ - data).ravel()
    #return (SOL - data).ravel()
    #sol_=sol[:, 11]
    return (sol_ - data).ravel()
    
    

In [159]:
initial_conditions = [initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V,initR3,initR4, initN, initD, initI_C]


beta = 0.31213499
beta1 =0.87152799
beta2 = 0.31231543
k1 = 0.25000000
k2 = 0.37000000
deta1 = 0.07987868
deta2 = 0.00996049

gama1 =0.20895232
gama2 = 0.31982875 
alpha1=0.01126056
yita1=6.3899e-04
yita2= 8.1267e-03


params['beta'].value = beta
params['beta1'].value = beta1
params['beta2'].value = beta2
params['k1'].value = k1
params['k2'].value = k2
params['deta1'].value = deta1
params['deta2'].value = deta2
params['gama1'].value = gama1
params['gama2'].value = gama2
params['alpha1'].value = alpha1
params['yita1'].value = yita1
params['yita2'].value = yita2
days = 140
tspan = np.arange(0, days, 1)


data=z

In [160]:
# fit model and find predicted values
result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq')
# result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq', \
# **{'xtol':1.e-15, 'ftol':1.e-15})

In [161]:
result.params

name,value,initial value,min,max,vary
beta,0.42605163,0.31213499,0.00000000,5.00000000,True
beta1,0.86071786,0.87152799,0.00000000,5.00000000,True
beta2,0.30808569,0.31231543,0.00000000,5.00000000,True
k1,0.25000000,0.25,0.15000000,0.25000000,True
k2,0.37000000,0.37,0.30000000,0.37000000,True
deta1,0.02857055,0.07987868,0.00000000,0.10000000,True
deta2,0.00999999,0.00996049,0.00000000,0.01000000,True
gama1,0.22511183,0.20895232,0.05000000,0.40000000,True
gama2,0.33101105,0.31982875,0.05000000,0.40000000,True
alpha1,0.00851883,0.01126056,0.00000000,0.10000000,True


In [162]:
# display fitted statistics
report_fit(result)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 14454
    # data points      = 149
    # variables        = 12
    chi-square         = 19307805.1
    reduced chi-square = 140932.884
    Akaike info crit   = 1778.03898
    Bayesian info crit = 1814.08633
##  Warning: uncertainties could not be estimated:
    k1:      at initial value
    k1:      at boundary
    k2:      at initial value
    k2:      at boundary
    deta2:   at boundary
    yita2:   at boundary
[[Variables]]
    beta:    0.42605163 (init = 0.312135)
    beta1:   0.86071786 (init = 0.871528)
    beta2:   0.30808569 (init = 0.3123154)
    k1:      0.25000000 (init = 0.25)
    k2:      0.37000000 (init = 0.37)
    deta1:   0.02857055 (init = 0.07987868)
    deta2:   0.00999999 (init = 0.00996049)
    gama1:   0.22511183 (init = 0.2089523)
    gama2:   0.33101105 (init = 0.3198287)
    alpha1:  0.00851883 (init = 0.01126056)
    yita1:   1.0126e-06 (init = 0.00063899)
    yita2:   8.9731e-13 (i

In [15]:
result.params.pretty_print()

Name       Value      Min      Max   Stderr     Vary     Expr Brute_Step
alpha1   0.01126        0      0.1     None     True     None     None
beta      0.3134        0        5     None     True     None     None
beta1     0.8691        0        5     None     True     None     None
beta2     0.3137        0        5     None     True     None     None
deta1    0.06588        0     0.08     None     True     None     None
deta2       0.01        0     0.01     None     True     None     None
gama1     0.2066     0.03     0.22     None     True     None     None
gama2       0.32     0.03     0.32     None     True     None     None
k1          0.25     0.15     0.25     None     True     None     None
k2          0.37      0.3     0.37     None     True     None     None
yita1       0.01        0     0.01     None     True     None     None
yita2   0.0009988        0    0.001     None     True     None     None


In [29]:
result.errorbars

True

In [5]:
from datetime import datetime
import matplotlib.dates as mdates


In [31]:
data1 = pd.read_excel('/Volumes/TOSHIBA EXT/per country variant.xlsx')
z=data1['s.a.real.d'][0:12]
x=data1['s.a.real.o'][0:12]
m=data1['s.a.fit.d'][0:12]
n=data1['s.a.fit.o'][0:12]
print(z)


0     0.988372
1     0.742574
2     0.066946
3     0.024516
4     0.012755
5     0.014925
6     0.011997
7     0.022901
8     0.017143
9     0.031128
10    0.022770
11    0.016287
Name: s.a.real.d, dtype: float64


In [32]:
data2 = pd.read_excel('/Volumes/TOSHIBA EXT/percountrydata/Denmark daily cases.xlsx')
z=data2['cases'][0:500]
x=data2['fit omicron'][0:500]
x1=data2['sim_eta2=0'][0:500]
x2=data2['sim_eta2=0.0003'][0:500]
x3=data2['sim_eta2=0.001'][0:500]
x4=data2['sim_beta1=2,eta2=0.001'][0:500]
x5=data2['sim_beta1=2,eta2=0.0003'][0:500]
x6=data2['sim_late_150,eta2=0.0003'][0:500]
x7=data2['sim_late_150,eta2=0.0001'][0:500]
y1=data2['pre_0.001'][0:22]
y2=data2['late-0.001'][0:22]
y3=data2['late-0.0001'][0:22]
y4=data2['late-0.001-beta1=2beta'][0:22]
y5=data2['mid-0.001'][0:22]
y=data2['late-fit'][0:22]
y0=data2['late-real'][0:22]
z1=data2['cmp,eta2=0.001'][0:300]
z2=data2['without-cmp'][0:300]
z3=data2['cmp,eta2=0.001,mid'][0:300]
z4=data2['cmp,beta1=2beta'][0:300]
z0=data2['standard'][0:300]


In [33]:

    
    fig = go.Figure()
    




    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=n, mode='lines', name='Fitting Omicron',line_shape='spline' ,line = dict ( width = 5 , color = 'LightSkyBlue' )))
    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=m, mode='lines', name='Fitting Non-Omicron',line_shape='spline',line = dict ( width = 5 , color = 'lightcoral' )))
  
   
    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=x, mode='markers',\
                             name='Omicron', line = dict(dash='dash')))
    fig.update_traces ( marker = dict ( size = 10 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))
    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=z, mode='markers',\
                             name='Non-Omicron', line = dict(dash='dash'),marker=dict ( color='lightcoral',
        size = 10 , line = dict ( width = 1 , color = 'black' ))))
        
        

    
    # Edit the layout
    fig.update_layout(font=dict(size=25),
                        
                       xaxis_title='Day',
                       yaxis_title='Percentage',
                       #yaxis_title='Total Cases',
                       title_x=0.5,width=900, height=600, template = 'simple_white',legend=dict(yanchor="top",y=0.85,xanchor="left",x=0.58,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>18 Oct 2021</b>','<b>15 Nov 2021</b>', '<b>13 Dec 2021</b>','<b>10 Jan 2022</b>', '<b>7 Feb 2022</b>', '<b>7 Mar 2022</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, 169, 28))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("images/sa1.p.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)

    
    fig.show()


In [8]:

    
    fig = go.Figure()
    




    fig.add_trace(go.Scatter(x=np.arange(0,71,14), y=n, mode='lines', name='Fitting BA.4/5',line_shape='spline' ,line = dict ( width = 5 , color = 'LightSkyBlue' )))
    fig.add_trace(go.Scatter(x=np.arange(0,71,14), y=m, mode='lines', name='Fitting Non-BA.4/5',line_shape='spline',line = dict ( width = 5 , color = 'lightcoral' )))
  
   
    fig.add_trace(go.Scatter(x=np.arange(0,71,14), y=x, mode='markers',\
                             name='BA.4/5', line = dict(dash='dash')))
    fig.update_traces ( marker = dict ( size = 10 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))
    fig.add_trace(go.Scatter(x=np.arange(0,71,14), y=z, mode='markers',\
                             name='Non-BA.4/5', line = dict(dash='dash'),marker=dict ( color='lightcoral',
        size = 10 , line = dict ( width = 1 , color = 'black' ))))
        
        

    
    # Edit the layout
    fig.update_layout(font=dict(size=25),
                        
                       xaxis_title='Day',
                       yaxis_title='Percentage',
                       #yaxis_title='Total Cases',
                       title_x=0.5,width=900, height=600, template = 'simple_white',legend=dict(yanchor="top",y=0.85,xanchor="left",x=0.58,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>21 Feb 2022</b>', '<b>21 Mar 2022</b>','<b>18 Apr 2022</b> ',  '<b>16 May 2022</b>', '<b>13 Jun 2022</b>', '<b>11 Jul 2022</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, 85, 14))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("images/sa.p.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)

    
    fig.show()


In [13]:

    
    fig = go.Figure()
    




    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=n, mode='lines', name='Fitting Omicron',line_shape='spline' ,line = dict ( width = 5 , color = 'LightSkyBlue' )))
    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=m, mode='lines', name='Fitting Non-Omicron',line_shape='spline',line = dict ( width = 5 , color = 'lightcoral' )))
  
   
    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=x, mode='markers',\
                             name='Omicron', line = dict(dash='dash')))
    fig.update_traces ( marker = dict ( size = 10 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))
    fig.add_trace(go.Scatter(x=np.arange(0,169,14), y=z, mode='markers',\
                             name='Non-Omicron', line = dict(dash='dash'),marker=dict ( color='lightcoral',
        size = 10 , line = dict ( width = 1 , color = 'black' ))))
        
        

    
    # Edit the layout
    fig.update_layout(font=dict(size=25),
                        
                       xaxis_title='Day',
                       yaxis_title='Percentage',
                       #yaxis_title='Total Cases',
                       title_x=0.5,width=900, height=600, template = 'simple_white',legend=dict(yanchor="top",y=0.85,xanchor="left",x=0.58,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>18 Oct 2021</b>','<b>15 Nov 2021</b>', '<b>13 Dec 2021</b>','<b>10 Jan 2022</b>', '<b>7 Feb 2022</b>', '<b>7 Mar 2022</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, 169, 28))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("images/sa1.p.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)

    
    fig.show()


In [34]:

    
    fig = go.Figure()
    




    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x, mode='lines', name='Fitting Omicron',line_shape='spline' ,line = dict ( width = 4 , color = 'lightcoral' )))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x1, mode='lines', name='\u03B7\u2082=0, \u03B2\u2081=1.5\u03B2',line_shape='spline',line = dict ( width = 4 , color = 'lightcoral',dash="dot" )))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x2, mode='lines', name='\u03B7\u2082=3\u00d710\u207B\u2074, \u03B2\u2081=1.5\u03B2',line_shape='spline',line = dict ( width = 4 , color = '#91B822',dash="dot" )))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x3, mode='lines', name='\u03B7\u2082=10\u207B\u00b3, \u03B2\u2081=1.5\u03B2',line_shape='spline',line = dict ( width = 4 , color = '#80D1C8')))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x4, mode='lines', name='\u03B7\u2082=10\u207B\u00b3, \u03B2\u2081=2\u03B2',line_shape='spline',line = dict ( width = 4 , color = '#FFE76F' )))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x5, mode='lines', name='\u03B7\u2082=3\u00d710\u207B\u2074, \u03B2\u2081=2\u03B2',line_shape='spline',line = dict ( width = 4 , color = '#FFD4A9',dash="dot" )))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x6, mode='lines', name='\u03B7\u2082=3\u00d710\u207B\u2074, Late invasion',line_shape='spline',line = dict ( width = 4 , color = '#8BA583' )))
    fig.add_trace(go.Scatter(x=np.arange(0,500,1), y=x7, mode='lines', name='\u03B7\u2082=10\u207B\u2074, Late invasion',line_shape='spline',line = dict ( width = 4 , color = '#CDA59E',dash="dot" )))
    
   
    '''fig.add_trace(go.Scatter(x=np.arange(0,240,1), y=x, mode='markers',\
                             name='Omicron', line = dict(dash='dash')))
    fig.update_traces ( marker = dict ( size = 10 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))'''
    fig.add_trace(go.Scatter(x=np.arange(0,180,1), y=z, mode='markers',\
                             name='Real data', line = dict(dash='dash'),marker=dict ( color='lightcoral',
        size = 5 , line = dict ( width = 1 , color = 'black' ))))
        
        

    
    # Edit the layout
    fig.update_layout(font=dict(size=25),
                        
                       xaxis_title='Day',
                       #yaxis_title='Percentage',
                       yaxis_title='Daily Cases',
                       title_x=0.5,width=1050, height=750, template = 'simple_white',legend=dict(font=dict(size=16.2),yanchor="top",y=0.998,xanchor="left",x=0.701,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>15 Nov 2021</b>', '<b>27 Feb 2022</b>','<b>07 Jun 2022</b>', '<b>14 Sep 2022</b>', '<b>22 Dec 2022</b>', '<b>31 Mar 2023</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, 500, 99))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("/Users/mac/Desktop/case&per/compare pre.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)

    
    fig.show()


In [413]:

    
    fig = go.Figure()
    




    fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=y, mode='lines', name='Fitting Omicron',line_shape='spline' ,line = dict ( width = 4 , color = 'lightcoral',dash="dot" )))

    fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=y1, mode='lines', name='\u03B7\u2082=10\u207B\u00b3, Early intvasion',line_shape='spline',line = dict ( width = 4 , color = '#8BA583' )))
    fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=y2, mode='lines', name='\u03B7\u2082=10\u207B\u00b3, Late invasion',line_shape='spline',line = dict ( width = 4 , color = 'LightSkyBlue' )))
    fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=y3, mode='lines', name='\u03B7\u2082=10\u207B\u2074, Late invasion',line_shape='spline',line = dict ( width = 4 , color = '#80D1C8',dash="dot")))
    fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=y4, mode='lines', name='\u03B7\u2082=10\u207B\u00b3, Late invasion, \u03B2\u2081=2\u03B2',line_shape='spline',line = dict ( width = 4 , color = '#FFE76F',dash="dot" )))
    fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=y5, mode='lines', name='\u03B7\u2082=10\u207B\u00b3, Midterm invasion',line_shape='spline',line = dict ( width = 4 , color = '#FFD4A9')))
    #fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=x6, mode='lines', name=r'$\eta_2=3 \times 10^{-4}, dely=150\ days$',line_shape='spline',line = dict ( width = 4 , color = '#8BA583' )))
    #fig.add_trace(go.Scatter(x=np.arange(0,21,1), y=x7, mode='lines', name=r'$\eta_2= 10^{-4}, dely=150\ days$',line_shape='spline',line = dict ( width = 4 , color = '#CDA59E',dash="dot" )))
    
   
    '''fig.add_trace(go.Scatter(x=np.arange(0,240,1), y=x, mode='markers',\
                             name='Omicron', line = dict(dash='dash')))
    fig.update_traces ( marker = dict ( size = 10 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))'''
    fig.add_trace(go.Scatter(x=np.arange(0,8,1), y=y0, mode='markers',\
                             name='Real data', line = dict(dash='dash'),marker=dict ( color='lightcoral',
        size = 8 , line = dict ( width = 1 , color = 'black' ))))
        
        

    
    # Edit the layout
    fig.update_layout(font=dict(size=25),
                        
                       xaxis_title='Day',
                       #yaxis_title='Percentage',
                       yaxis_title='Percentage',
                       title_x=0.5,width=950, height=650, template = 'simple_white',legend=dict(font=dict(size=20),yanchor="top",y=0.575,xanchor="left",x=0.54,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>15 Nov 2021</b>', '<b>27 Feb 2022</b>','<b>07 Jun 2022</b>', '<b>14 Sep 2022</b>', '<b>22 Dec 2022</b>', '<b>31 Mar 2023</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, 21, 4))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    fig.write_image("/Users/mac/Desktop/case&per/compare replace time.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)

    
    fig.show()


In [36]:

    
    fig = go.Figure()
    




    #fig.add_trace(go.Scatter(x=np.arange(0,300,1), y=z0, mode='lines', name='Fitting Omicron',line_shape='spline' ,line = dict ( width = 4 , color = 'lightcoral' )))
    fig.add_trace(go.Scatter(x=np.arange(0,260,1), y=z1, mode='lines', name='\u03B2\u2081=1.5\u03B2, Early invasion',line_shape='spline',line = dict ( width = 4 , color = '#FFD4A9' )))
    fig.add_trace(go.Scatter(x=np.arange(0,260,1), y=z2, mode='lines', name='without competition',line_shape='spline',line = dict ( width = 4 , color = '#91B822' )))
    fig.add_trace(go.Scatter(x=np.arange(0,260,1), y=z3, mode='lines', name='\u03B2\u2081=1.5\u03B2, Midterm invasion',line_shape='spline',line = dict ( width = 4 , color = '#80D1C8')))
    fig.add_trace(go.Scatter(x=np.arange(0,260,1), y=z4, mode='lines', name='\u03B2\u2081=2\u03B2, Early invasion',line_shape='spline',line = dict ( width = 4 , color = 'lightcoral' )))
    #fig.add_trace(go.Scatter(x=np.arange(0,300,1), y=x5, mode='lines', name=r'$\eta_2=3 \times 10^{-4}, \beta_1=2\beta$',line_shape='spline',line = dict ( width = 4 , color = '#FFD4A9',dash="dot" )))
    #fig.add_trace(go.Scatter(x=np.arange(0,300,1), y=x6, mode='lines', name=r'$\eta_2=3 \times 10^{-4}, dely=150\ days$',line_shape='spline',line = dict ( width = 4 , color = '#8BA583' )))
    #fig.add_trace(go.Scatter(x=np.arange(0,300,1), y=x7, mode='lines', name=r'$\eta_2= 10^{-4}, dely=150\ days$',line_shape='spline',line = dict ( width = 4 , color = '#CDA59E',dash="dot" )))
    
   
    '''fig.add_trace(go.Scatter(x=np.arange(0,240,1), y=x, mode='markers',\
                             name='Omicron', line = dict(dash='dash')))
    fig.update_traces ( marker = dict ( size = 10 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))'''
    fig.add_trace(go.Scatter(x=np.arange(0,150,1), y=z, mode='markers',\
                             name='Real data', line = dict(dash='dash'),marker=dict ( color='lightcoral',
        size = 5 , line = dict ( width = 1 , color = 'black' ))))
        
        

    
    # Edit the layout
    fig.update_layout(font=dict(size=25),
                        
                       xaxis_title='Day',
                       #yaxis_title='Percentage',
                       yaxis_title='Daily Cases',
                       title_x=0.5,width=950, height=650, template = 'simple_white',legend=dict(font=dict(size=20),yanchor="top",y=0.998,xanchor="left",x=0.585,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>15 Nov 2021</b>', '<b>18 Jan 2022</b>','<b>19 Mar 2022</b>', '<b>17 May 2022</b>', '<b>15 Jul 2022</b>', '<b>31 Mar 2023</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, 260, 60))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("/Users/mac/Desktop/case&per/compare influnce.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)

    
    fig.show()


In [3]:
df = pd.read_excel('/Users/mac/Desktop/model/community_net_dist.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mac/Desktop/model/community_net_dist.xlsx'

In [ ]:
color_names = [’’, ‘Jan’, ‘Feb’, ‘Mar’, ‘Apr’, ‘May’, ‘June’, ‘July’, ‘Aug’, ‘Sept’]
color_vals = list(range(len(color_names)))
num_colors = len(color_vals)

In [4]:
import pandas as pd
import plotly.express as px
import random

# create data frame

x0=[0.03,1.2,5.4,9.8,13,17,18,31,45]
y0=[9,8,7,6,5,4,3,2,1]
Rank=[77,60,35,33,26,18,9,3,2.5]
size1=[4585978,3619465,2158386,2036536,1559115,1162533,564721,202217,156823]
#TEXT= ['77%','60%','35%','33%','26%','18%','9%','3%','2.5%']
# create scatterplot
fig = px.scatter(df, y=y0, x=x0,color=Rank,size=size1,color_continuous_scale='sunsetdark',size_max=70)

# move legend
fig.update_layout(font=dict(size=20),
                        
                       xaxis_title='Immunity scale',
                       #yaxis_title='Number of New Cases',
                       yaxis_title='Rank',legend=dict(y=1, x=0.1),width=900, height=500,template = 'simple_white',coloraxis_colorbar=dict(
    title="Final size (%) ",
    #thicknessmode="pixels", thickness=50,
    #lenmode="pixels", len=430,
    #yanchor="top", y=1.05,
    #ticks="outside", ticksuffix=" bills",
    #dtick=5
))
fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['0%', '5%','10% ', '15%','20%', '25%', '30%','35% ', '40%','45%'],tickformat = None, tickmode='array',tickvals=np.arange(0, 46, 5))

fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=[0,'<b>$Denmark$</b>','<b>$South$ $Korea$</b> ','<b>$Bahrain$</b>','<b>$UK$</b>','<b>$Spain$</b>','<b>$US$</b> ','<b>$Malaysia$</b> ','<b>$South$ $Africa$</b>','<b>$China$</b>'],tickformat = None, tickmode='array',tickvals=np.arange(0, 9.5, 1))
#fig.write_image("/Users/mac/Desktop/case&per/09.JPG",scale=2)
#fig.write_image("images/m.png", format="png")    
fig.show()

/opt/anaconda3/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning:

Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message



NameError: name 'df' is not defined